In [ ]:
stop_words = [
        "a", "about", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along",
        "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
        "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are", "around", "as", "at", "back", "be",
        "became", "because", "become", "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
        "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom", "but", "by", "call", "can", "co",
        "con", "could", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight",
        "either", "eleven", "else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
        "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill", "find", "fire", "first", "five", "for",
        "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had",
        "has", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself",
        "him", "himself", "his", "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed", "interest",
        "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made",
        "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much",
        "must", "my", "myself", "name", "namely", "neither", "nevertheless", "next", "nine", "nobody", "now", "nowhere",
        "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours",
        "ourselves", "out", "over", "own", "part", "per", "perhaps", "please", "put", "rather", "re", "same", "see",
        "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some",
        "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take",
        "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby",
        "therefore", "therein", "thereupon", "these", "they", "thick", "thin", "third", "this", "those", "though",
        "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve",
        "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what",
        "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon",
        "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will",
        "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves"
        ]

In [ ]:
# Function to calculate the accuracy of our predictions vs labels
def predict_classwise_stance_emergent(P_relatedness, P_stance, stance_labels):

    P_related = torch.reshape(P_relatedness[:, 0], (-1, 1))
    P_unrelated = torch.reshape(P_relatedness[:, 1], (-1, 1))
    
    tmp1 = P_stance[:,:3]
    tmp2 = torch.reshape(torch.sum(tmp1,dim=1),[-1,1])
    tmp3 = torch.cat([tmp2,tmp2,tmp2],dim=1)
    tmp4 = torch.cat([P_related, P_related, P_related],dim=1)
    tmp5 = torch.div(tmp1,tmp3)
    tmp6 = tmp5*tmp4
    prob = torch.cat([tmp6,P_unrelated],1)#tmp6
    
    target_labels = torch.argmax(torch.abs(stance_labels), 1)
    predict_labels = torch.argmax(prob, 1)


    
    agree_true = 0
    agree_total = 0
    
    disagree_true = 0
    disagree_total = 0
    
    discuss_true = 0
    discuss_total = 0
    
    unrelated_true = 0
    unrelated_total = 0
    
    for idx, true_label in enumerate(target_labels):
        predict_label = predict_labels[idx] 
        if true_label == 0:
            agree_total += 1
            if predict_label == 0:
                agree_true += 1
        elif true_label == 1:
            disagree_total += 1
            if predict_label == 1:
                disagree_true += 1
        elif true_label == 2:
            discuss_total += 1
            if predict_label == 2:
                discuss_true += 1
        elif true_label == 3:
            unrelated_total += 1
            if predict_label == 3:
                unrelated_true += 1
    accuracy_agree = 0
    if agree_true != 0:
        accuracy_agree = agree_true/agree_total
        
    accuracy_disagree = 0
    if disagree_true != 0:
        accuracy_disagree = disagree_true/disagree_total
    
    accuracy_discuss = 0
    if discuss_true != 0:
        accuracy_discuss = discuss_true/discuss_total
        
    accuracy_notrelated = 0
    if unrelated_true != 0:
        accuracy_notrelated = unrelated_true/unrelated_total
        
    true_predict_count = len((torch.eq(predict_labels, target_labels)).nonzero().flatten())
    accuracy = true_predict_count / len(predict_labels)
    
    true_total = agree_true + disagree_true + discuss_true + unrelated_true
    #print("*************")
    #print("Total True")
    #print(true_total)
    
    return accuracy, accuracy_agree, accuracy_disagree, accuracy_discuss, accuracy_notrelated, true_total

In [ ]:
def preprocess_data_train(df, dfVal, dfTest, lim_unigram):
    
    id_ref = {}
    claims_t = []    
    headlines_t = []
    headlines_track_t = {}
    
    for index, instance in df.iterrows():
        claim = instance['claimHeadline']

    
    for index, instance in df.iterrows():
        claim = instance['claimHeadline']      
        headline = instance['articleHeadline']
             
        if headline not in headlines_track_t:
            headlines_t.append(headline)
            headlines_track_t[headline] = 1
        if claim not in claims_t:
            claims_t.append(claim)
            
    claims_v = []
    headlines_v = []
    headlines_track_v = {}
    for index, instance in dfVal.iterrows():
        claim = instance['claimHeadline']        
        headline = instance['articleHeadline']
        
        if headline not in headlines_track_v:
            headlines_v.append(headline)
            headlines_track_v[headline] = 1
        if claim not in claims_v:
            claims_v.append(claim)            

      
    ####################
            
    claims_test = []  
    headlines_test = []
    headlines_track_test = {}
    for index, instance in dfTest.iterrows():
        claim = instance['claimHeadline']
        headline = instance['articleHeadline']
            
        if headline not in headlines_track_test:
            headlines_test.append(headline)
            headlines_track_test[headline] = 1
        
        if claim not in claims_test:
            claims_test.append(claim)

            
        # Create reference dictionary
    for i, elem in enumerate(headlines_t + claims_t):
        id_ref[elem] = i

        
    # Create vectorizers and BOW and TF arrays for train set
    bow_vectorizer = CountVectorizer(max_features=lim_unigram, stop_words=stop_words)
    bow = bow_vectorizer.fit_transform(claims_t + headlines_t)  # Train set only
    
    tfreq_vectorizer = TfidfTransformer(use_idf=False).fit(bow)
    tfreq = tfreq_vectorizer.transform(bow).toarray()  # Train set only
    
    tfidf_vectorizer = TfidfVectorizer(max_features=lim_unigram, stop_words=stop_words).\
        fit(headlines_t + claims_t + headlines_v + claims_v + headlines_test + claims_test)  # Train and test sets
    
    ####################
    
    
    headline_tfidf_track = {}
    claim_tfidf_track = {}
    headline_all = []
    claim_all = []
    cos_track = {}
    
    train_set = []
    print(len(df))
    
    for index, instance in df.iterrows():
        headline = instance['articleHeadline']
        claim = instance['claimHeadline']
        
        headline_all.append(headline)
        claim_all.append(claim)
        
        headline_tf = tfreq[id_ref[headline]].reshape(1, -1)
        claim_tf = tfreq[id_ref[claim]].reshape(1, -1)
        
        if headline not in headline_tfidf_track:
            head_tfidf = tfidf_vectorizer.transform([headline]).toarray()
            headline_tfidf_track[headline] = head_tfidf
        else:
            head_tfidf = headline_tfidf_track[headline]
            
        if claim not in claim_tfidf_track:
            claim_tfidf = tfidf_vectorizer.transform([claim]).toarray()
            claim_tfidf_track[claim] = claim_tfidf
        else:
            claim_tfidf = claim_tfidf_track[claim]
            
        if (headline, claim) not in cos_track:
            tfidf_cos = cosine_similarity(head_tfidf, claim_tfidf)[0].reshape(1, 1)
            cos_track[(headline, claim)] = tfidf_cos
        else:
            tfidf_cos = cos_track[(headline, claim)]

        feat_vec = np.squeeze(np.c_[headline_tf, claim_tf, tfidf_cos])
        train_set.append(feat_vec)

    X_overlap = gen_or_load_feats(word_overlap_features, headline_all, claim_all, 'features_fnc_onlytrain/train_overlap.npy')
    X_refuting = gen_or_load_feats(refuting_features, headline_all, claim_all, 'features_fnc_onlytrain/train_refuting.npy')
    X_polarity = gen_or_load_feats(polarity_features, headline_all, claim_all, 'features_fnc_onlytrain/train_polarity.npy')
    X_hand = gen_or_load_feats(hand_features, headline_all, claim_all, 'features_fnc_onlytrain/train_hand.npy')
    
    train_features = np.squeeze(np.c_[X_refuting, X_polarity,X_overlap,X_hand])#
    train_set = np.squeeze(np.c_[train_set,train_features])
    ####### preprocessing
    train_set = np.asarray(train_set)
    train_mean = np.mean(train_set, axis = 0)
    train_set = train_set-train_mean
    
    return train_set, train_mean, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer

In [ ]:
def preprocess_data_val(dfVal, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer, m):
    
    headlines_track = {}
    claims_track = {}
    
    headline_all = []
    claim_all = []
    cos_track = {}
    
    test_set = []
    
    for index, instance in dfVal.iterrows():
        headline = instance['articleHeadline']
        claim = instance['claimHeadline']
        headline_all.append(headline)
        claim_all.append(claim)
        
        if headline not in headlines_track:
            head_bow = bow_vectorizer.transform([headline]).toarray()
            headline_tf = tfreq_vectorizer.transform(head_bow).toarray()[0].reshape(1, -1)
            headline_tfidf = tfidf_vectorizer.transform([headline]).toarray().reshape(1, -1)
            headlines_track[headline] = (headline_tf, headline_tfidf)
        else:
            headline_tf = headlines_track[headline][0]
            headline_tfidf = headlines_track[headline][1]
            
        if claim not in claims_track:
            claim_bow = bow_vectorizer.transform([claim]).toarray()
            claim_tf = tfreq_vectorizer.transform(claim_bow).toarray()[0].reshape(1, -1)
            claim_tfidf = tfidf_vectorizer.transform([claim]).toarray().reshape(1, -1)
            claims_track[headline] = (claim_tf, claim_tfidf)
        else:
            claim_tf = claims_track[claim][0]
            claim_tfidf = claims_track[claim][1]
            
        if (headline, claim) not in cos_track:
            tfidf_cos = cosine_similarity(headline_tfidf, claim_tfidf)[0].reshape(1, 1)
            cos_track[(headline, claim)] = tfidf_cos
        else:
            tfidf_cos = cos_track[(headline, claim)]
            
        feat_vec = np.squeeze(np.c_[headline_tf, claim_tf, tfidf_cos])
        test_set.append(feat_vec)
        
        
    X_overlap = gen_or_load_feats(word_overlap_features, headline_all, claim_all, 'features_fnc_onlytrain/val_overlap.npy')
    X_refuting = gen_or_load_feats(refuting_features, headline_all, claim_all, 'features_fnc_onlytrain/val_refuting.npy')
    X_polarity = gen_or_load_feats(polarity_features, headline_all, claim_all, 'features_fnc_onlytrain/val_polarity.npy')
    X_hand = gen_or_load_feats(hand_features, headline_all, claim_all, 'features_fnc_onlytrain/val_hand.npy')
    
    test_features = np.squeeze(np.c_[X_refuting,X_polarity,X_overlap,X_hand])#
    test_set = np.squeeze(np.c_[test_set, test_features])
    ####### preprocessing
    test_set = np.asarray(test_set)
    test_set = (test_set-m)#/(std+0.0001)
    
    return test_set

In [ ]:
def preprocess_data_test(dfTest, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer, m):
    
    headlines_track = {}
    claims_track = {}
    
    headline_all = []
    claim_all = []
    cos_track = {}
    
    test_set = []
    
    for index, instance in dfTest.iterrows():
        headline = instance['articleHeadline']
        claim = instance['claimHeadline']
        headline_all.append(headline)
        claim_all.append(claim)
        
        if headline not in headlines_track:
            head_bow = bow_vectorizer.transform([headline]).toarray()
            headline_tf = tfreq_vectorizer.transform(head_bow).toarray()[0].reshape(1, -1)
            headline_tfidf = tfidf_vectorizer.transform([headline]).toarray().reshape(1, -1)
            headlines_track[headline] = (headline_tf, headline_tfidf)
        else:
            headline_tf = headlines_track[headline][0]
            headline_tfidf = headlines_track[headline][1]
            
        if claim not in claims_track:
            claim_bow = bow_vectorizer.transform([claim]).toarray()
            claim_tf = tfreq_vectorizer.transform(claim_bow).toarray()[0].reshape(1, -1)
            claim_tfidf = tfidf_vectorizer.transform([claim]).toarray().reshape(1, -1)
            claims_track[headline] = (claim_tf, claim_tfidf)
        else:
            claim_tf = claims_track[claim][0]
            claim_tfidf = claims_track[claim][1]
            
        if (headline, claim) not in cos_track:
            tfidf_cos = cosine_similarity(headline_tfidf, claim_tfidf)[0].reshape(1, 1)
            cos_track[(headline, claim)] = tfidf_cos
        else:
            tfidf_cos = cos_track[(headline, claim)]
            
        feat_vec = np.squeeze(np.c_[headline_tf, claim_tf, tfidf_cos])
        test_set.append(feat_vec)
        
        
    X_overlap = gen_or_load_feats(word_overlap_features, headline_all, claim_all, 'features_fnc_onlytrain/test_overlap.npy')
    X_refuting = gen_or_load_feats(refuting_features, headline_all, claim_all, 'features_fnc_onlytrain/test_refuting.npy')
    X_polarity = gen_or_load_feats(polarity_features, headline_all, claim_all, 'features_fnc_onlytrain/test_polarity.npy')
    X_hand = gen_or_load_feats(hand_features, headline_all, claim_all, 'features_fnc_onlytrain/test_hand.npy')
    
    test_features = np.squeeze(np.c_[X_refuting,X_polarity,X_overlap,X_hand])#
    test_set = np.squeeze(np.c_[test_set, test_features])
    ####### preprocessing
    test_set = np.asarray(test_set)
    test_set = (test_set-m)#/(std+0.0001)
    
    return test_set

In [ ]:
def load_dataset_stance_fnc(path):
    df = pd.read_csv(path, delimiter='\t', header = 0, names=['BodyID', 'claimHeadline', 'articleHeadline', 'stance'])
    
    return df

In [ ]:
def preprocess_fnc(labels):
    t_relatedness = []
    t_stance = []
    
    t_mmd_symbol = []
    t_mmd_symbol_ = []
    
    #print(labels.shape)
    for label in labels:
        if label == 3: #unrelated
            t_relatedness.append([0,1])
            t_stance.append([0,0,0,-1])
            t_mmd_symbol.append(0)
            t_mmd_symbol_.append(1)
        elif label == 1: #disagree
            t_relatedness.append([1,0])
            t_stance.append([0,1,0,0])
            t_mmd_symbol.append(1)
            t_mmd_symbol_.append(0)
        elif label == 0: #agree
            t_relatedness.append([1,0])
            t_stance.append([1,0,0,0])
            t_mmd_symbol.append(1)
            t_mmd_symbol_.append(0)       
        elif label == 2: #discuss
            t_relatedness.append([1,0])
            t_stance.append([0,0,1,0])
            t_mmd_symbol.append(1)
            t_mmd_symbol_.append(0)

    t_relatedness = torch.as_tensor(t_relatedness, dtype=torch.int32)
    t_stance = torch.as_tensor(t_stance, dtype=torch.int32)
    
    t_mmd_symbol  = torch.as_tensor(t_mmd_symbol, dtype=torch.float32)
    t_mmd_symbol_ = torch.as_tensor(t_mmd_symbol_, dtype=torch.float32)
    
    return t_relatedness, t_stance, t_mmd_symbol, t_mmd_symbol_

In [ ]:
def preprocess_fnc_test(labels):
    t_relatedness = []
    t_stance = []
    
    t_mmd_symbol = []
    t_mmd_symbol_ = []
    
    #print(labels.shape)
    for label in labels:
        if label == 3: #unrelated
            t_relatedness.append([0,1])
            t_stance.append([0,0,0,1])
            t_mmd_symbol.append(0)
            t_mmd_symbol_.append(1)
        elif label == 1: #disagree
            t_relatedness.append([1,0])
            t_stance.append([0,1,0,0])
            t_mmd_symbol.append(1)
            t_mmd_symbol_.append(0)
        elif label == 0: #agree
            t_relatedness.append([1,0])
            t_stance.append([1,0,0,0])
            t_mmd_symbol.append(1)
            t_mmd_symbol_.append(0)       
        elif label == 2: #discuss
            t_relatedness.append([1,0])
            t_stance.append([0,0,1,0])
            t_mmd_symbol.append(1)
            t_mmd_symbol_.append(0)

    t_relatedness = torch.as_tensor(t_relatedness, dtype=torch.int32)
    t_stance = torch.as_tensor(t_stance, dtype=torch.int32)
    
    t_mmd_symbol  = torch.as_tensor(t_mmd_symbol, dtype=torch.float32)
    t_mmd_symbol_ = torch.as_tensor(t_mmd_symbol_, dtype=torch.float32)
    
    return t_relatedness, t_stance, t_mmd_symbol, t_mmd_symbol_

In [ ]:
#Generate the datasets with the different fields.
def generate_datasets_fnc(df):
    
    sentencesQuery = df.Q.values #claim in this case
    labels = df.stance.values

    sentencesCont = df.docCont.values

    #*****Experiment 1a: Title and docCont: maxLen - 64 - different with the default case
    sentencesQueryCont = concanListStringsLonger(sentencesQuery, sentencesCont)

    return sentencesQueryCont, labels

In [ ]:
import torch
from transformers import BertModel, RobertaModel, DistilBertModel
class StanceDetectionUnigramClass(torch.nn.Module):
    def __init__(self, datasetUsed):
        super(StanceDetectionUnigramClass, self).__init__()
        input_size = len(datasetUsed[0])
        hidden_size_initial = 100
        hidden_size = 100
        mmd_size = 10
        dropout_prob = 0.6
        dropout_prob2 = 0.6
        relatedness_size = 2
        classes_size = 4
        ideology_class_size = 3
        #agreement_size = 3
        #self.input_pl = BertForPreTraining.from_pretrained(modelUsed) #input
        #self.input_pl = BertModel.from_pretrained(modelUsed)
        #self.input_pl = RobertaModel.from_pretrained(modelUsed)
        #self.input_pl = DistilBertModel.from_pretrained(modelUsed)
        self.l1 = torch.nn.Linear(input_size, hidden_size_initial)
        self.l2 = torch.nn.Linear(hidden_size_initial, hidden_size)
        self.l3 = torch.nn.Linear(hidden_size, hidden_size)
        self.bn1_hidden = torch.nn.BatchNorm1d(hidden_size_initial, momentum=0.05)
        self.bn2_hidden = torch.nn.BatchNorm1d(hidden_size, momentum=0.05)
        self.dropout = torch.nn.Dropout(dropout_prob)
        self.dropout2 = torch.nn.Dropout(dropout_prob2)

        self.theta_d = torch.nn.Linear(hidden_size, mmd_size)
        self.bn1_theta = torch.nn.BatchNorm1d(mmd_size, momentum=0.05)
        
        self.probability = torch.nn.Linear(hidden_size, relatedness_size)
        self.output_prob = torch.nn.Softmax(dim = 1)
        
        self.stance = torch.nn.Linear(hidden_size + relatedness_size - 1, classes_size)
        self.ideology = torch.nn.Linear(hidden_size + classes_size - 2, ideology_class_size)
        
        #for param in self.input_pl.embeddings.parameters():
            #param.requires_grad = False
        
        #for param in self.input_pl[2][0:5].parameters():
            #param.requires_grad = False

        #self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, mmd_pl, mmd_pl_):
        relatedness_size = 2
        classes_size = 4
        ideology_class_size = 3
        
        #hidden layer
        hidden_state = self.l1(input_ids)
        hidden_state_normalized = self.bn1_hidden(hidden_state)
        hidden_state_normalized = torch.nn.ReLU()(hidden_state_normalized)
        hidden_layer= self.dropout2(hidden_state_normalized)
    
        #mmd layer        
        theta_d = self.theta_d(hidden_layer)
        theta_d_normalized = self.bn1_theta(theta_d)
        theta_d_normalized = torch.nn.ReLU()(theta_d_normalized)
        theta_d_layer= self.dropout2(theta_d_normalized)
        
        
        n1 = torch.sum(mmd_pl, dim = 0) + 1e-10
        n2 = torch.sum(mmd_pl_, dim = 0)  + 1e-10
        aa = torch.reshape(mmd_pl, (-1,1))
        bb = torch.reshape(mmd_pl_, (-1,1))
        
        #calculate mmd_loss                  
        d1 = torch.div(torch.sum(torch.mul(theta_d_layer, aa), dim=1), n1)
        d2 = torch.div(torch.sum(torch.mul(theta_d_layer, bb), dim=1), n2)
                             
        mmd_loss = torch.sum(d1 - d2)

        #probability layer
        relatedness_state = self.probability(hidden_layer)
        relatedness_flat = self.dropout2(relatedness_state)
        
        relatedness_flat_reshaped = torch.reshape(relatedness_flat, (-1, relatedness_size))
        P_relatedness = self.output_prob(relatedness_flat_reshaped)
        #P_relatedness = relatedness_flat_reshaped
        
        P_related = torch.reshape(P_relatedness[:, 0], (-1, 1))
        P_unrelated = torch.reshape(P_relatedness[:, 1], (-1, 1))
        
        #stance layer
        concat_fea = torch.cat([hidden_layer, P_related], dim = 1)
        stance_state = self.stance(concat_fea) #batch size x classes_size
        stance_flat = self.dropout2(stance_state) #batch size x classes_size
        
        stance_flat_reshaped = torch.reshape(stance_flat, (-1, classes_size))
        P_stance = self.output_prob(stance_flat_reshaped) 
        

        return mmd_loss, P_relatedness, P_stance

In [ ]:
#from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
def prepare_for_training_stance_emergent_unigram(instancesTrain, labelsTrain, instancesVal, labelsVal, modelUsed, batch_size=16, epochs = 50, num_warmup_steps=0, learning_rate=5e-5):
    # Combine the training inputs into a TensorDataset.

    from transformers import BertForSequenceClassification, AdamW, BertConfig, RobertaConfig, AutoModelWithLMHead
    from transformers import DistilBertForSequenceClassification, RobertaForSequenceClassification
    
    from torch.utils.data import DataLoader, RandomSampler
    
    
    t_train_relatedness, t_train_stance, t_train_mmd_symbol, t_train_mmd_symbol_ = preprocess_fnc(labelsTrain)
    t_instancesTrain  = torch.as_tensor(instancesTrain, dtype=torch.float32)
    datasetTrain = TensorDataset(t_instancesTrain, t_train_relatedness, t_train_stance, t_train_mmd_symbol, t_train_mmd_symbol_)

    # Combine the training inputs into a TensorDataset.
    t_val_relatedness, t_val_stance, t_val_mmd_symbol, t_val_mmd_symbol_ = preprocess_fnc_test(labelsVal)
    t_instancesVal = torch.as_tensor(instancesVal, dtype=torch.float32)
    datasetVal = TensorDataset(t_instancesVal, t_val_relatedness, t_val_stance, t_val_mmd_symbol, t_val_mmd_symbol_)
    
    
    model = StanceDetectionUnigramClass(instancesTrain)
    

    #print(model)
    # Tell pytorch to run this model on the GPU.
    #model.cuda()

    # Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
    # I believe the 'W' stands for 'Weight Decay fix"
    
    
    optimizer = AdamW(model.parameters(),
                  lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  betas=(0.9, 0.999), 
                  eps=1e-08, 
                  weight_decay=1e-5,
                  correct_bias=True
               )

    train_dataloader = DataLoader(
            datasetTrain,  # The training samples.
            sampler =  RandomSampler(datasetTrain), # Select batches randomly
            batch_size = batch_size, # Trains with this batch size., 
            num_workers=8
        )
    batch_size = batch_size


    from transformers import get_linear_schedule_with_warmup, get_cosine_with_hard_restarts_schedule_with_warmup

    # Number of training epochs. The BERT authors recommend between 2 and 4. 
    # We chose to run for 4, but we'll see later that this may be over-fitting the
    # training data.
    epochs = epochs

    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(train_dataloader) * epochs

    # Create the learning rate scheduler.
    schedulerOld = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = num_warmup_steps, # Default value in run_glue.py
                                            num_training_steps = total_steps)
    
    scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(optimizer, num_warmup_steps = num_warmup_steps, # Default value in run_glue.py
                                            num_training_steps = total_steps, num_cycles = 5)
    
    return model, datasetTrain, datasetVal, optimizer, schedulerOld

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler
#from tensorboardX import SummaryWriter

#import EarlyStopping
def onlytrain_stance_emergent_unigram(model_save_path, model, tokenizer, datasetTrain, datasetVal, epochs, batch_size, optimizer, scheduler, patience, verbose, delta, seedVal, continue_train = False):
    writer = SummaryWriter()
    min_val_loss = 100
    max_val_acc = 0
    
    relatedness_size = 2
    classes_size = 4
    loss_fct_relatedness = torch.nn.BCEWithLogitsLoss()
    
    loss_fct_stance = torch.nn.CrossEntropyLoss()
    #loss_fct = torch.nn.BCEWithLogitsLoss()
    
    alpha = 1.5
    beta = 1e-3
    theta = 0
    gamma = 0
    
    batch_size_max_once = batch_size

    if batch_size < batch_size_max_once:
        batch_size_max_once = batch_size
        
    accumulation_steps = batch_size/batch_size_max_once
    
    writer = SummaryWriter()

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()
    train_dataloader, validation_dataloader = return_batches_datasets(datasetTrain, datasetVal, batch_size_max_once)
    
    epoch_start = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        #multi-gpu
        if torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
            model = torch.nn.DataParallel(model)
            
    print(device)
    
    
            
    if continue_train:
        checkpoint = torch.load('./model_save/fnc/model_fncbert.t7')
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch_start = checkpoint['epoch']
    
    torch.cuda.empty_cache()
    model.to(device)
    optimizer_to(optimizer,device)
    
    
     #pos_weight=torch.FloatTensor ([28.36 / 0.5090]
    
     #pos_weight = torch.tensor([1.0, 1.0, 1.0])
     #pos_weight = pos_weight.to(device)
     #criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    weights_ideology = torch.tensor([10, 5, 1.5]).to(device)   
    weights_stance = torch.tensor([0.82, 0.85, 0.67, 0.3]).to(device) 
    loss_fct_relatedness_weighted = torch.nn.BCEWithLogitsLoss(pos_weight = weights_stance)
    
    # For each epoch...
    batch_epoch_count = 1
    for epoch_i in range(epoch_start, epoch_start + epochs):
        
        print("---------Epoch----------" + str(epoch_i))
        
        # ========================================
        #               Training
        # ========================================
    
        # Perform one full pass over the training set.

        #print("")
        #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        #print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0
        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()
        model.zero_grad()
        optimizer.zero_grad()
        # For each batch of training data...
        mini_batch_avg_loss = 0
        #train_size = len(train_dataloader)
        
        if batch_epoch_count % 500 == 0:
            batch_size = batch_size*2
            accumulation_steps = int(batch_size/batch_size_max_once)
        batch_epoch_count = batch_epoch_count + 1

        #train_size = len(train_dataloader) / float(accumulation_steps)
        
        print("Batch Size: " + str(batch_size))
        print(float(accumulation_steps))
        
        #print("Learning rate: ", scheduler.get_last_lr())
        for step, batch in enumerate(train_dataloader):
            
            with torch.autograd.detect_anomaly():
                elapsed = format_time(time.time() - t0)
        
                b_input_ids = batch[0].to(device)
                b_relatedness = batch[1].to(device)
                b_labels = batch[2].to(device)
                b_mmd_symbol = batch[3].to(device)
                b_mmd_symbol_ = batch[4].to(device)
        
            
            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            

                mmd_loss, P_relatedness, P_stance = model(input_ids = b_input_ids, mmd_pl = b_mmd_symbol, mmd_pl_ = b_mmd_symbol_)

            
                relatedness_loss = loss_fct_relatedness(P_relatedness, b_relatedness.float())
                stance_loss = loss_fct_relatedness(P_stance, b_labels.float())

            
                loss = relatedness_loss + alpha * stance_loss - beta * mmd_loss
                total_train_loss += loss.item()
            
            
                model.zero_grad()
                optimizer.zero_grad()
                loss.backward()

                #for parameter in model.parameters(): print(parameter.grad.norm())

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                
                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                optimizer.step()

                # Update the learning rate.
                scheduler.step()
                
                #for param_group in optimizer.param_groups:
                #print("Learning Rate: ", optimizer.param_groups["lr"])
                
                                
                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because 
                # accumulating the gradients is "convenient while training RNNs". 
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                model.zero_grad()
                optimizer.zero_grad()
                #total_train_loss = 0

                #print("Iter: %02d, Loss: %4.4f" % (step, mini_batch_avg_loss))                
    
        print("Learning rate: ", scheduler.get_last_lr())
        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)
    
        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        #print("")
        print("  Average training loss: {0:.6f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        
        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.
        model_save_state = {
            'epoch': epoch_i + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
            }
        
        torch.save(model_save_state, model_save_path)

    last_epoch = 1
    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

    return training_stats, last_epoch, min_val_loss, max_val_acc

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler
#from tensorboardX import SummaryWriter

#import EarlyStopping
def train_stance_emergent_unigram(model_save_path, model, tokenizer, datasetTrain, datasetVal, epochs, batch_size, optimizer, scheduler, patience, verbose, delta, seedVal, continue_train = False):
    writer = SummaryWriter()
    min_val_loss = 100
    
    relatedness_size = 2
    classes_size = 4
    loss_fct_relatedness = torch.nn.BCEWithLogitsLoss()
    
    loss_fct_stance = torch.nn.CrossEntropyLoss()
    #loss_fct = torch.nn.BCEWithLogitsLoss()
    
    alpha = 1.5
    beta = 1e-3
    theta = 0
    gamma = 0
    
    batch_size_max_once = batch_size

    if batch_size < batch_size_max_once:
        batch_size_max_once = batch_size
        
    accumulation_steps = batch_size/batch_size_max_once
    
    es = EarlyStopping(patience,verbose, delta)
    writer = SummaryWriter()

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()
    train_dataloader, validation_dataloader = return_batches_datasets(datasetTrain, datasetVal, batch_size_max_once)
    
    epoch_start = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    if torch.cuda.is_available():
        #multi-gpu
        if torch.cuda.device_count() > 1:
            print("Let's use", torch.cuda.device_count(), "GPUs!")
            # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
            model = torch.nn.DataParallel(model)
            
    print(device)
    
    
            
    if continue_train:
        checkpoint = torch.load('./model_save/fnc/model_fncbert.t7')
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        epoch_start = checkpoint['epoch']
    
    torch.cuda.empty_cache()
    model.to(device)
    optimizer_to(optimizer,device)
    
    
     #pos_weight=torch.FloatTensor ([28.36 / 0.5090]
    
     #pos_weight = torch.tensor([1.0, 1.0, 1.0])
     #pos_weight = pos_weight.to(device)
     #criterion = torch.nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    weights_ideology = torch.tensor([10, 5, 1.5]).to(device)   
    weights_stance = torch.tensor([0.82, 0.85, 0.67, 0.3]).to(device) 
    loss_fct_relatedness_weighted = torch.nn.BCEWithLogitsLoss(pos_weight = weights_stance)
    
    # For each epoch...
    batch_epoch_count = 1
    for epoch_i in range(epoch_start, epoch_start + epochs):
        
        print("---------Epoch----------" + str(epoch_i))
        
        # ========================================
        #               Training
        # ========================================
    
        # Perform one full pass over the training set.

        #print("")
        #print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        #print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0
        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()
        model.zero_grad()
        optimizer.zero_grad()
        # For each batch of training data...
        mini_batch_avg_loss = 0
        #train_size = len(train_dataloader)
        
        if batch_epoch_count % 500 == 0:
            batch_size = batch_size*2
            accumulation_steps = int(batch_size/batch_size_max_once)
        batch_epoch_count = batch_epoch_count + 1

        #train_size = len(train_dataloader) / float(accumulation_steps)
        
        print("Batch Size: " + str(batch_size))
        print(float(accumulation_steps))
        
        #print("Learning rate: ", scheduler.get_last_lr())
        for step, batch in enumerate(train_dataloader):
            
            with torch.autograd.detect_anomaly():
                elapsed = format_time(time.time() - t0)
        
                b_input_ids = batch[0].to(device)
                b_relatedness = batch[1].to(device)
                b_labels = batch[2].to(device)
                b_mmd_symbol = batch[3].to(device)
                b_mmd_symbol_ = batch[4].to(device)
        
            
            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            

                mmd_loss, P_relatedness, P_stance = model(input_ids = b_input_ids, mmd_pl = b_mmd_symbol, mmd_pl_ = b_mmd_symbol_)

            
                relatedness_loss = loss_fct_relatedness(P_relatedness, b_relatedness.float())
                stance_loss = loss_fct_relatedness(P_stance, b_labels.float())

            
                loss = relatedness_loss + alpha * stance_loss - beta * mmd_loss
                total_train_loss += loss.item()
            
            
                model.zero_grad()
                optimizer.zero_grad()
                loss.backward()

                #for parameter in model.parameters(): print(parameter.grad.norm())

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
                torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
                
                # Update parameters and take a step using the computed gradient.
                # The optimizer dictates the "update rule"--how the parameters are
                # modified based on their gradients, the learning rate, etc.
                optimizer.step()

                # Update the learning rate.
                scheduler.step()
                
                #for param_group in optimizer.param_groups:
                #print("Learning Rate: ", optimizer.param_groups["lr"])
                
                                
                # Always clear any previously calculated gradients before performing a
                # backward pass. PyTorch doesn't do this automatically because 
                # accumulating the gradients is "convenient while training RNNs". 
                # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
                model.zero_grad()
                optimizer.zero_grad()
                #total_train_loss = 0

                #print("Iter: %02d, Loss: %4.4f" % (step, mini_batch_avg_loss))                
    
        print("Learning rate: ", scheduler.get_last_lr())
        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)
    
        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        #print("")
        print("  Average training loss: {0:.6f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))
        
        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        #print("")
        #print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy_stance = 0
        total_eval_accuracy_ideology = 0
        total_eval_loss = 0
        nb_eval_steps = 0
        
        agree_test_accuracy = 0
        disagree_test_accuracy = 0 
        discuss_test_accuracy = 0 
        unrelated_test_accuracy = 0
        
        con_test_acc = 0
        lib_test_acc = 0
        na_test_acc = 0
        
        total_true = 0
        

        # Evaluate data for one epoch
        for batch in validation_dataloader:
        
            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_relatedness = batch[1].to(device)
            b_labels = batch[2].to(device)
            b_mmd_symbol = batch[3].to(device)
            b_mmd_symbol_ = batch[4].to(device)
        
            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
            
                mmd_loss, P_relatedness, P_stance = model(input_ids = b_input_ids, mmd_pl = b_mmd_symbol, mmd_pl_ = b_mmd_symbol_)
                
                #CrossEntropy Loss
                relatedness_loss = loss_fct_relatedness(P_relatedness, b_relatedness.float())
                stance_loss = loss_fct_relatedness(P_stance, b_labels.float())

                
    
                loss_val = relatedness_loss + alpha * stance_loss - beta * mmd_loss
                total_eval_loss += loss_val.item()

                # Move logits and labels to CPU
                P_relatedness = P_relatedness.to('cpu')
                b_relatedness = b_relatedness.to('cpu')
                P_stance = P_stance.to('cpu')
                b_labels = b_labels.to('cpu')
                

                # Calculate the accuracy for this batch of test sentences, and
                # accumulate it over all batches.
                #total_eval_accuracy += predict(P_relatedness, P_stance, b_labels)
                
                
                acc_list = predict_classwise_stance_emergent(P_relatedness, P_stance, b_labels)
                total_eval_accuracy_stance += acc_list[0]
                ###
                agree_test_accuracy += acc_list[1]
                disagree_test_accuracy += acc_list[2]
                discuss_test_accuracy += acc_list[3]
                unrelated_test_accuracy += acc_list[4]
                total_true += acc_list[5]
        

        # Report the final accuracy for this validation run.
        avg_val_accuracy_stance = total_eval_accuracy_stance / len(validation_dataloader)
        print("Avg Val Accuracy Stance: {0:.6f}".format(avg_val_accuracy_stance))
        print("Total True")
        print(total_true)
        print("*************")
        avg_val_agree_accuracy = agree_test_accuracy / len(validation_dataloader)
        print("Avg Val Agree Accuracy: {0:.6f}".format(avg_val_agree_accuracy))
        avg_val_disagree_accuracy = disagree_test_accuracy / len(validation_dataloader)
        print("Avg Val Disagree Accuracy: {0:.6f}".format(avg_val_disagree_accuracy))
        avg_val_discuss_accuracy = discuss_test_accuracy / len(validation_dataloader)
        print("Avg Val Discuss Accuracy: {0:.6f}".format(avg_val_discuss_accuracy))
        avg_val_unrelated_accuracy = unrelated_test_accuracy / len(validation_dataloader)
        print("Avg Val Unrelated Accuracy: {0:.6f}".format(avg_val_unrelated_accuracy))

        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)
    
        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)
        
        if avg_val_loss < min_val_loss:
            min_val_loss = avg_val_loss
    
        print("Avg Validation Loss: {0:.6f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))

        # Record all statistics from this epoch.
        training_stats.append(
            {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Stance Accur.': avg_val_accuracy_stance,
            'Training Time': training_time,
            'Validation Time': validation_time
            }
        )
        
        model_save_state = {
            'epoch': epoch_i + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
            }
    
        es.__call__(avg_val_loss, avg_val_accuracy_stance, model_save_state, model_save_path, model, tokenizer)
        last_epoch = epoch_i + 1
        if es.early_stop == True:
            break  # early stop criterion is met, we can stop now

    print("")
    print("Training complete!")

    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
    
    
    min_val_loss = es.val_loss_min
    max_val_acc = es.val_acc_max_stance

    return training_stats, last_epoch, min_val_loss, max_val_acc

In [ ]:
from torch.utils.data import DataLoader, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig

def run_test_stance(model_savepath, testData, stance_labels_Test, batch_size):    
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    #loss_fct = torch.nn.BCELoss()
    loss_fct_relatedness = torch.nn.BCEWithLogitsLoss()
    
    t_test_relatedness, t_test_stance, t_test_mmd_symbol, t_test_mmd_symbol_ = preprocess_fnc_test(stance_labels_Test)
    t_instancesTest  = torch.as_tensor(testData, dtype=torch.float32)

    

    # Create the DataLoader.
    prediction_data = TensorDataset(t_instancesTest, t_test_relatedness, t_test_stance, t_test_mmd_symbol, t_test_mmd_symbol_)
    #prediction_data = TensorDataset(all_input_ids_Test, all_input_masks_Test, t_test_stance)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size, num_workers=8, drop_last=True)
    
    #model_current = 'bert-base-uncased'
    model_current = 'roberta-large'
    tokenizer = load_tokenizer(model_current)
        
    model = StanceDetectionUnigramClass(testData)
    checkpoint = torch.load(model_savepath)
    model.load_state_dict(checkpoint['state_dict'])    
    
    optimizer = AdamW(model.parameters(),
                  lr = learning_rate, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  betas=(0.9, 0.999), 
                  eps=1e-08, 
                  weight_decay=0,
                  correct_bias=True
    )
    optimizer.load_state_dict(checkpoint['optimizer'])
    epoch_start = checkpoint['epoch']
    
    torch.cuda.empty_cache()
    model.to(device)
    optimizer_to(optimizer,device)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model.to(device)
    
    #model.cuda()
    # Put model in evaluation mode
    model.eval()

    # Tracking variables
    total_test_loss = 0.0
    
    total_test_accuracy = 0.0
    ####
    agree_test_accuracy = 0.0
    disagree_test_accuracy = 0.0
    discuss_test_accuracy = 0.0
    unrelated_test_accuracy = 0.0
    predictions , true_labels = [], []
    
    alpha = 1.5
    theta = 1.1
    beta = 1e-3
    # Predict 
    for batch in prediction_dataloader:
      #Add batch to GPU
        
        #batch = tuple(t.to(device) for t in batch)
        
        b_input_ids = batch[0].to(device)
        b_relatedness = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_mmd_symbol = batch[3].to(device)
        b_mmd_symbol_ = batch[4].to(device)
  


        # Telling the model not to compute or store gradients, saving memory and 
        # speeding up prediction
        with torch.no_grad():         
            # Forward pass, calculate logit predictions
            
            n1 = torch.sum(b_mmd_symbol, dim=0)
            n2 = torch.sum(b_mmd_symbol_, dim=0)
        
            aa = torch.reshape(b_mmd_symbol, (-1,1))
            bb = torch.reshape(b_mmd_symbol_, (-1,1))
            
            theta_d_layer, P_relatedness, P_stance = model(input_ids = b_input_ids, mmd_pl = b_mmd_symbol, mmd_pl_ = b_mmd_symbol_)
            #P_stance = model(input_ids = b_input_ids, attention_mask = b_input_mask)
                
            if n1 == 0:
                d1 = torch.zeros(batch_size, 1, device = device)
            else:
                d1 = torch.div(torch.sum(theta_d_layer*aa, dim=1), n1)
                
            if n2 == 0:
                d2 = torch.zeros(batch_size, 1, device = device)
            else:
                d2 = torch.div(torch.sum(theta_d_layer*bb, dim=1), n2)
                    
                    
            mmd_loss = torch.sum(d1 - d2)
            #mmd_loss = 0
                
            
            relatedness_loss = loss_fct_relatedness(P_relatedness, b_relatedness.float())
            #relatedness_loss = 0
            stance_loss = loss_fct_relatedness(P_stance, b_labels.float())
                
    
            loss_test = relatedness_loss + alpha * stance_loss - beta * mmd_loss
            #loss_test = alpha * stance_loss
            total_test_loss += loss_test.item()
            
            # Move logits and labels to CPU
            P_relatedness = P_relatedness.to('cpu')
            b_relatedness = b_relatedness.to('cpu')
            P_stance = P_stance.to('cpu')
            b_labels = b_labels.to('cpu')
            
            acc_list = predict_classwise_stance_emergent(P_relatedness, P_stance, b_labels)
            total_test_accuracy += acc_list[0]
            ###
            agree_test_accuracy += acc_list[1]
            disagree_test_accuracy += acc_list[2]
            discuss_test_accuracy += acc_list[3]
            unrelated_test_accuracy += acc_list[4]
            

    # Report the final accuracy for this validation run.
    avg_test_loss = total_test_loss / len(prediction_dataloader)
    avg_test_accuracy = total_test_accuracy / len(prediction_dataloader)
    
    avg_agree_test_acc = agree_test_accuracy / len(prediction_dataloader)
    avg_disagree_test_acc = disagree_test_accuracy / len(prediction_dataloader)
    avg_discuss_test_acc = discuss_test_accuracy / len(prediction_dataloader)
    avg_unrelated_test_acc = unrelated_test_accuracy / len(prediction_dataloader)

    return avg_test_loss, avg_test_accuracy, avg_agree_test_acc, avg_disagree_test_acc, avg_discuss_test_acc, avg_unrelated_test_acc

In [ ]:
def run_wholeprocess_fnc_unigram(train_path, val_path, max_len, doc_stride, batch_size, num_warmup_steps, learning_rate, seedVal):
    device = run_utils()
    model_save_path = './model_save/emergent/model_emergentbert.t7'  
    #model_fnc = './model_save/fnc/model_fnc.t7'    
        
    #model_current = 'distilbert-base-uncased'
    #model_current = 'roberta-base'
    model_current = 'bert-base-uncased'
    #model_current = './models/tiny_bert/'
    tokenizer = load_tokenizer(model_current)

#--------------LOAD DATASETS--------------#
    lim_unigram = 3000
    
    train_path = './dataset/fnc/train_fnc.csv'
    val_path = './dataset/fnc/val_fnc.csv'
    test_path = './dataset/fnc/test_fnc.csv'

    df = load_dataset_stance_fnc(train_path)
    dfVal = load_dataset_stance_fnc(val_path)
    dfTest = load_dataset_stance_fnc(test_path)
    
    df = df.append(dfVal, ignore_index = True)
    
    # Report the number of sentences.
    print('Number of training sentences: {:,}'.format(df.shape[0]))
    print('Number of val sentences: {:,}'.format(dfVal.shape[0]))
    print('Number of test sentences: {:,}'.format(dfTest.shape[0]))
    
    train_set, train_mean, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer = preprocess_data_train(df, dfVal, dfTest, lim_unigram)
    val_set = preprocess_data_val(dfVal, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer, m = 0.0001)
    test_set = preprocess_data_test(dfTest, bow_vectorizer, tfreq_vectorizer, tfidf_vectorizer, m = 0.0001)

    labelsTrain = df['stance']
    labelsVal = dfVal['stance']
    labelsTest = dfTest['stance']
    #--------------TRAINING-------------#
    
    model, train_dataloader, validation_dataloader, optimizer, scheduler = prepare_for_training_stance_emergent_unigram(train_set, labelsTrain, val_set, labelsVal, model_current, 16, epochs, num_warmup_steps, learning_rate)    
    training_stats, last_epoch, min_val_loss, max_val_acc = onlytrain_stance_emergent_unigram (model_save_path, model, tokenizer, train_dataloader, validation_dataloader, epochs, batch_size, optimizer,
                                                                          scheduler, patience, verbose, delta, seedVal, False)
    
    test_loss, test_acc, avg_agree_test_acc, avg_disagree_test_acc, avg_discuss_test_acc, avg_unrelated_test_acc = run_test_stance(model_save_path, test_set, labelsTest, batch_size)
        
    #df_stats = print_summary(training_stats)
    #plot_results(df_stats, last_epoch)

    print('Min Val Loss: ' + str(min_val_loss))
    print('Max Val Acc: ' + str(max_val_acc))
    
    print('Test Loss: ' + str(test_loss))
    print('Test Stance Acc: ' + str(test_acc))
    
    print('Test Agree Class Acc: ' + str(avg_agree_test_acc))
    print('Test Disagree Class Acc: ' + str(avg_disagree_test_acc))
    print('Test Discuss Class Acc: ' + str(avg_discuss_test_acc))
    print('Test Unrelated Class Acc: ' + str(avg_unrelated_test_acc))

In [ ]:
!pip install import_ipynb
!pip install nltk
import nltk
nltk.download('popular')

In [ ]:
import os
import string
import pandas as pd
import numpy as np
import random
from random import randint
import time
from transformers import AutoModel
from transformers import DistilBertModel
from torch.utils.data import TensorDataset, random_split
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import import_ipynb
import feature_engineering
from feature_engineering import *
from Utils import *


import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#model = "bert-base-uncased"
train_path = './dataset/fnc/train'
val_path = './dataset/fnc/test'

model_save_path = './model_save'


max_len = 512
doc_stride = 0

batch_size = 64
epochs = 1
num_warmup_steps = 0
learning_rate = 1e-5
##-----Early Stopping
patience = 10
verbose = True
delta = 0.000001

value = randint(0, 100)
seedVal = 7

#seedVal = 40

#0 - tinybert
#1 - distilbert
#2 - bert

#bert_type = 2

#create_determinism(seedVal)

#folder_preparations()
run_wholeprocess_fnc_unigram(train_path, val_path, max_len, doc_stride, batch_size, num_warmup_steps, learning_rate, seedVal)
#create_more_notrel_docs()